In [1]:
#
# importing the necessary libraries: 
# NumPy, SciPy (spatial algorithms), timing functions, a handler for csv files 
#
import numpy as np
import scipy.spatial as sp
import time
import csv

In [2]:
#
# Centering a point cloud A
#
def barycentered(A):
    #
    bar = np.sum(A, axis=1)/A.shape[1]
    #
    return A - bar[:, np.newaxis]

In [3]:
#
# Best orthogonal fit between two (centred) labelled point clouds
#
# A and B are assumed to have the same number of points
#
def best_fit(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na==nb
    #
    H = A @ B.T
    W, S, V = np.linalg.svd(H)
    #
    return V.T @ W.T

In [4]:
#
# Detecting nearest neighbours between two unlabelled point coluds A and B
#
# Assumption: cardinality of A <= cardinality of B 
#
# Input: point clouds A, B
#
# Output: for each point of A a matching point of B, as a 0/1-matrix
#
def nearest_neighbours(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na <= nb
    #
    tree = sp.KDTree(A.T, leafsize=10, compact_nodes=True, copy_data=True, balanced_tree=True)
    #
    matching = []
    I = matrix.identity(na)
    #
    for i in range(nb):
        _, ind = tree.query(B.T[i], k=1, p=2, workers=-1)
        matching += [I[ind]]
    #    
    return np.array(matching)

In [5]:
#
# ICP algorithm for two unlabelled point clouds A and B
#
# Assumption: cardinality of A <= cardinality of B
#
# Input:
# A, B = unlabelled point clouds
# init = initial orthogonal transformation
# max_iter = maximum number of iterations
# tol = tolerance for halting the computation
#
# Output:
# U = orthogonal transformation bringing A to B as close as possible
# nn = nearest neighbor matching between U*A and B
# dist = distance between U*A and B*nn (in the max singular value norm)
#
def icp(A, B, init=None, max_iter=100, tol=1e-16):
    #
    src = copy(A)
    dst = copy(B)
    #
    na = src.shape[1]
    nb = src.shape[1]
    #
    assert na <= nb
    #
    if (init is not None):
        src = init @ src
    #    
    prev_err = 0
    #
    for i in range(max_iter):
        #   
        nn = nearest_neighbours(src, dst)
        U = best_fit(src, dst @ nn)
        #    
        src = U @ src
        #
        diff = src - dst @ nn
        err  = np.linalg.norm(diff, 2)
        #
        if abs(prev_err - err) < tol:
            break
        #    
        prev_err = err
    #
    U = best_fit(A, src)
    nn = nearest_neighbours(U @ A, dst)
    #
    diff = U @ A - B @ nn
    dist = np.linalg.norm(diff, 2)
    #
    return U, nn, dist

In [6]:
#
# Testing the Main Algorithm on a random point cloud generated as a dim * num matrix
# with uniformly distributed entries in the range [min_coord, max_coord]
#
# Generates random P, O and S, and produces Q = O*P*S, then applies the Main Algorithm
#
# Outputs flag = True if o = O and s = S are successfully recovered, flag = False otherwise
#
# Prints out o, s, and checks their proximity to O, S, respectively
# Prints the distance between Q and o*P*s in the max singular value norm
#
def test_random(dim=3, num=100, min_coord=-20, max_coord=20, verbose=False):
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]    
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    P = np.random.uniform(min_coord, max_coord, (dim, num))
    Q = O @ P @ S
    #
    P = barycentered(P)
    Q = barycentered(Q)
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    assert np.allclose(O @ Ep @ O.T - Eq, np.zeros([dim,dim]))
    assert np.allclose(U0 @ Ep @ U0.T - Eq, np.zeros([dim,dim]))
    assert(np.allclose(Eigp, Eigq))
    #
    isoms_discrete = MatrixGroup([matrix.diagonal(d) for d in Permutations([-1]+[1]*(dim-1))])
    isoms_discrete = [np.array(matrix(m)) for m in isoms_discrete]
    #
    flag = False
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        o, s, d = icp(P, Q, U)
        flag = flag or np.allclose(d, 0)
        if flag:
            if verbose:
                print("Orthogonal transformation found:")
                print(o)
                print("Equal to the initial one?")
                print(np.allclose(o, O))
                print("Permutation found:")
                print(s.T)
                print("Equal to the initial one?")
                print(np.allclose(s.T, S))
                print("Distance to image:")
                print(d)
            break
    #
    return flag

In [7]:
#
# Running a given number of tests num_tests on random point clouds 
# generated by calling test_random(dim, num, min_coord, max_coord)
#
# Some statistics is collected (time elapsed, test success rate, etc)
#
def run_tests_random(num_tests, dim=3, num=100, min_coord=-20, max_coord=20, verbose=False):
    #
    num_success = 0
    num_fail  = 0
    #
    start = time.process_time()
    #
    for i in range(num_tests):
        #
        msg = '### Test #{} : '.format(i+1)
        if verbose:
            print(msg)
        #
        test_flag = test_random(dim, num, min_coord, max_coord, verbose)
        #
        if test_flag:
            num_success += 1
        else:
            num_fail += 1
        #
        msg = msg + ' SUCCESS {}'.format(num_success)
        msg = msg + ' FAIL {}'.format(num_fail)
        print(msg, end='\r')
    #
    end = time.process_time()
    #
    assert (num_success + num_fail == num_tests)
    #
    print("Time elapsed:", time.strftime('%H:%M:%S', time.gmtime(end-start)), ' '*42)
    print("Success rate:", float(num_success/num_tests))

In [8]:
#
# Running 100 tests in dim 3 with random point clouds each having 100 points
#
run_tests_random(num_tests=100, dim=3, num=100, min_coord=-20, max_coord=20, verbose=False)

Time elapsed: 00:05:44                                           
Success rate: 1.0


In [9]:
#
# Testing the Main Algorithm on point clouds that are not randomly generated
#

In [10]:
#
# Raghupathi, Sunand; Brunhart-Lupo, Nicholas; Gruchalla, Kenny (2020)
# "Caerbannog Point Clouds". National Renewable Energy Laboratory. 10.7799/1729892
# Source: https://data.nrel.gov/submissions/153
#

In [11]:
#
# Testing the Main Algorithm on a given point cloud representing an image
#
# Input: a point cloud P
#
# Generates a random O and S, and produces Q = O*P*S, then applies the Main Algorithm
#
# Output: flag = True if o = O and s = S are successfully recovered, flag = False otherwise
#
# Prints out o, s, and checks their proximity to O, S, respectively
# Prints the distance between Q and o*P*s in the max singular value norm
#
def test_point_cloud(P, verbose=False):
    #
    dim = P.shape[0]
    num = P.shape[1]
    if verbose:
        print("Number of points: {}".format(num))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    P = barycentered(P)
    #
    Q = O @ P @ S
    Q = barycentered(Q)
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    assert np.allclose(O @ Ep @ O.T - Eq, np.zeros([dim,dim]))
    assert np.allclose(U0 @ Ep @ U0.T - Eq, np.zeros([dim,dim]))
    assert(np.allclose(Eigp, Eigq))
    #
    isoms_discrete = MatrixGroup([matrix.diagonal(d) for d in Permutations([-1]+[1]*(dim-1))])
    isoms_discrete = [np.array(matrix(m)) for m in isoms_discrete]
    #
    flag = False
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        o, s, d = icp(P, Q, U)
        flag = flag or np.allclose(d, 0)
        if flag:
            if verbose:
                print("Orthogonal transformation found:")
                print(o)
                print("Equal to the initial one?")
                print(np.allclose(o, O))
                print("Permutation found:")
                print(s.T)
                print("Equal to the initial one?")
                print(np.allclose(s.T, S))
                print("Distance to image:")
                print(d)
            break
    #
    return flag

In [12]:
#
# Running a given number of tests num_tests on various point clouds 
# generated by calling test_point_cloud(P, verbose)
#
# Some statistics is collected (time elapsed, test success rate, etc)
#
def run_tests_point_cloud(P, num_tests, verbose=False):
    #
    num_success = 0
    num_fail  = 0
    #
    start = time.process_time()
    #
    for i in range(num_tests):
        #
        msg = '### Test #{} : '.format(i+1)
        if verbose:
            print(msg)
        #
        test_flag = test_point_cloud(P, verbose)
        #
        if test_flag:
            num_success += 1
        else:
            num_fail += 1
        #
        msg = msg + ' SUCCESS {}'.format(num_success)
        msg = msg + ' FAIL {}'.format(num_fail)
        print(msg, end='\r')
    #   
    end = time.process_time
    #
    assert (num_success + num_fail == num_tests)
    #
    print("Time elapsed:", time.strftime('%H:%M:%S', time.gmtime(end-start)), ' '*42)
    print("Success rate:", float(num_success/num_tests))

In [13]:
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
#
filenames = ["Teapot.csv", "Bunny.csv", "Cow.csv"]
#
for name in filenames:
    #
    P = []
    #
    f = open(name)
    reader = csv.reader(f)
    #
    for line in reader:
        P += [[RDF(v) for v in line]]
    #
    f.close()
    #    
    P = np.array(P).T
    #
    print("File read: {}".format(name))
    #
    run_tests_point_cloud(P, num_tests=100, verbose=False)
    #
    print("#"*42)

File read: Teapot.csv
Time elapsed: 00:49:05                                           
Success rate: 1.0
##########################################
File read: Bunny.csv
Time elapsed: 02:11:03                                           
Success rate: 1.0
##########################################
File read: Cow.csv
Time elapsed: 01:37:40                                           
Success rate: 1.0
##########################################
